# DISCOVERING AND ASSIGNING TOPICS FOR ADVOCACY CAMPAIGN PETITION MAILINGS USING LATENT DIRICHLET ALLOCATION (LDA)

### In this phase of my project I use machine learning models to cluster SumOfUs mailings into issue topics. Then, for each topic, I assign a percentage to each campaign, based on the probability that topic relates to that campaign.  The algorthim I am using is called Latent Dirichlet Allocation (LDA).  

### The LDA algoritm will process the text of 1399 SumOfUs advocacy mailings and discover the top 8 issues, or topics, in SumOfUs campaigns.  The LDA will then go back through each mailing and assign it a percentage for each of the 8 topics.  Some campaigns pertain only to one topic, while others might relate to several topics in differing degreees.

### After I assign the percentages to each campaign, I will go on to the next phase, where I will combine this information with other data about the campaign, such as virality, regional distribution, and the very early behavior of new joiners on that campaign.  All of this will go into a different model, called a logistic regression, that will help me to predict the donation propensity of a new cohort only a week after joining the list.

In [1]:
#import the first Python modules I will be using in the project
import pandas as pd  #for working with data in table form within Python
import numpy as np   #important math and logic functions
from bs4 import BeautifulSoup # a module that works with HTML
import nltk  # import the Natural Language Toolkit, which includes various tools for text analysis
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models # LDA model
import pyLDAvis.gensim #visualization

### First, I read in the CSV file that I prepared in the Data Wrangling phase.  The file contains the page id, the name of each mailing tag, and the full HTML in each mailing. Since each mailing can have more than one tag, most mailings  have multiple rows in the file.

In [2]:
camp_txt = pd.read_csv('../capstone/page_mailing_selected.csv', encoding = "ISO-8859-1")  #import CSV as a Pandas table
camp_txt.tail(10)  #displays the last 10 rows of the table

,page_id,tag_name,html
2092,16154,#Environment,<div style=width: 320px; float: right;>\r\n<ta...
2093,16148,#Environment,<div style=width: 320px; float: right;>\r\n<ta...
2094,16160,bees,<div style=width: 320px; float: right;>\r\n<ta...
2095,16160,#Environment,<div style=width: 320px; float: right;>\r\n<ta...
2096,16416,#Environment,<p>Dear {{ user.first_name|capfirst|default:Fr...
2097,16458,#Workers_Rights,<div style=width: 320px; float: right;>\r\n<ta...
2098,16479,#Workers_Rights,<div style=width: 320px; float: right;>\r\n<ta...
2099,16736,#Privatization_and_Political_Meddling,<div style=width: 320px; float: right;>\r\n<ta...
2100,16940,oil company,<table style=width: 100%; max-width: 300px; ma...
2101,16940,#Environment,<table style=width: 100%; max-width: 300px; ma...


### In order for my program to work, I need each mailing to be in a single row, not multiple rows for every tag. I do this by taking every tag for a single mailing and turning it into a list.  Then I can 'flatten' the current table.  Now, instead of 2101 rows, my table only has 1399 rows, one for each mailing.

In [33]:
flat = pd.DataFrame(camp_txt)  #copy the current dataframe into a new one (I found this helpful for troubleshooting)

def flatten_frame(df,col):  #df= DataFrame, col= the column to be flattened; in our case, 'tag_name'
    df[col] = df[col].fillna('') #if there are no tags, add an empty string
    headers = list(df.columns.values) #pull in the list of columns
    group_cols = list(set(headers) - set([col])) #get the cols to group on by subtracting the one we are flattening
    df = pd.DataFrame(df.groupby(by=(group_cols))[col].apply(list)).reset_index() #group by all the columns except the tags
    df[col] = df[col].apply(lambda x: ', '.join(x)) #convert the flattened col of tags into a string
    return df

flat = flatten_frame(flat,'tag_name') # executes the function defined above
flat = flat[['page_id','tag_name','html']].sort_values(by='page_id')  #sorts the new flattened table
flat = flat.reset_index(drop=True) #needed so the index keeps the same order of the page_ids

pd.options.display.max_colwidth = 110
flat.tail(10)  #displays the last 10 rows of the table

,page_id,tag_name,html
1390,16102,#Womens_Rights,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1391,16118,"taxes, #Economic_Justice, #trade",<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1392,16148,#Environment,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1393,16154,#Environment,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1394,16160,"bees, #Environment",<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1395,16416,#Environment,<p>Dear {{ user.first_name|capfirst|default:Friend }}</p>\r\n<p><strong>Water issues are sure heating up a...
1396,16458,#Workers_Rights,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1397,16479,#Workers_Rights,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1398,16736,#Privatization_and_Political_Meddling,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=0 cellspacing=0...
1399,16940,"oil company, #Environment",<table style=width: 100%; max-width: 300px; margin-left: 10px; margin-right: 10px; border=0 cellspacing=0 ...


### Now each campaign is on a single row, but in order to analyze the mailing text, I have to extract plain text from all that ugly HTML/Django. I do this with the help of a module called 'Beautiful Soup'. I also have a separate function that takes out all the punctuation from both the html and tag_name fields; I will need that done before the next stage of the analysis.

In [34]:
clean = pd.DataFrame(flat) #copy the results from the last step into a new frame

def clean_soup(df,old_col,new_col): # df=DataFrame, col=the column with the dirty HTML we want to clean
    for index, item in df[old_col].iteritems(): #go row by row through the column
        soup = BeautifulSoup(item, "lxml") #turn the current item into a BeautfulSoup object
        washed = soup.get_text(" ",strip=True) #get text from the soup object and store the text in the washed variable
        df.set_value(index,new_col,washed) #update the clean data frame with the washed text
    df[new_col] = df[new_col].str.replace('{(.+)}', ' ') #remove the django tags
    return df

def remove_punc(df,old_col,new_col):
    df[new_col] = df[old_col].str.replace('[^\w\s]',' ') #replaces most punctuation with spaces
    df[new_col] = df[new_col].str.replace('[_]',' ') #replaces underscores with spaces    
    return df

clean['text_clean'] ='' #a new column for storing our squeaky-clean text
clean['tags_clean'] = '' #a new column for storing our squeaky-clean tags
 
clean = clean_soup(clean,'html','text_clean')  #run my function to clean the html
clean = remove_punc(clean,'text_clean','text_clean')  #run my function to remove punctuation from the text
clean = remove_punc(clean,'tag_name','tags_clean') #run my function to premove punctuation from the tags

pd.options.display.max_colwidth = 95
clean[['page_id','html','text_clean']].tail(10)

,page_id,html,text_clean
1390,16102,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Craigslist allow exploitative adverts offering homeless women accommodation for sex Hundre...
1391,16118,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Apple is avoiding more than 13 billion in taxes And that s not even the half of it Deman...
1392,16148,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,The world s two biggest greenhouse gas polluters have now ratified the Paris Agreement so...
1393,16154,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,The world s two biggest greenhouse gas polluters have now ratified the Paris Agreement so...
1394,16160,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Regulators are rolling out a controversial pesticide banned in Europe to kill Zika mosquito...
1395,16416,<p>Dear {{ user.first_name|capfirst|default:Friend }}</p>\r\n<p><strong>Water issues are su...,Dear Great news After 225000 of us spoke out against the mismanagement of the precious w...
1396,16458,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Foreign worker lists That s what this government wants to make all firms publish Show tha...
1397,16479,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Asos is coming under fire for treating its workers like machines Tell Asos to treat its ...
1398,16736,<div style=width: 320px; float: right;>\r\n<table style=width: 300px; margin: 10px; border=...,Corporations have the government s ear when it comes to Brexit it promised Nissan tariff ...
1399,16940,<table style=width: 100%; max-width: 300px; margin-left: 10px; margin-right: 10px; border=0...,Urgent The Trudeau government could approve the Kinder Morgan tar sands pipeline in three ...


### Finally we have clear, readable text! However, not all of these words are salient for my purposes; they won't help my algothtm figuring out which campaigns belong to which topics. 

### At this point, I need to start whittling down this text to the words that will be most helpful for our analysis. On thing I consider is the part of speech.  Things like prepositions and conjunctions will confuse the LDA when it tries to dicern discrete topics.  Verbs and adverbs are largely unhelpful too.

### A Python module called the Natural Language Toolkit (NLTK) will help mw filter all the words in the mailing by part of speech.  After some trial and error, I decide to keep only nouns and plural nouns.  I do not inclde proper nouns, which strips out the names of the corporations we target. 

In [35]:
filtered = pd.DataFrame(clean[['page_id','text_clean','tags_clean']]) #copy relevant results of last step into a new frame

def filter_camp_by_pos(df,old_col,new_col,pos_codes):
    df[new_col] = df.apply(lambda row: nltk.word_tokenize(row[old_col]), axis=1) #tokenize (pre-process) each word
    df[new_col] = df[new_col].apply(lambda row: nltk.pos_tag(row)) #tag each word with the part of speech
    df[new_col] = [[tuple[0].lower() for tuple in row if tuple[1] in pos_codes] for row in df[new_col]]  #select all words matching my filter codes
    return df

pos_codes = ['NN','NNS']  #just nouns and plural nouns

filter_camp_by_pos(filtered,'text_clean','text_filtered',pos_codes)  #runs the function I defined about

pd.options.display.max_colwidth = 80
filtered[['page_id','text_clean','text_filtered']].tail(10)  #displays last 10 rows

,page_id,text_clean,text_filtered
1390,16102,Craigslist allow exploitative adverts offering homeless women accommodation ...,"[adverts, women, accommodation, sex, hundreds, vile, ads, petition, bans, pe..."
1391,16118,Apple is avoiding more than 13 billion in taxes And that s not even the ha...,"[taxes, half, tax, evasion, petition, secret, technology, giant, profits, go..."
1392,16148,The world s two biggest greenhouse gas polluters have now ratified the Paris...,"[world, s, greenhouse, gas, polluters, excuse, fossil, fuel, industry, clima..."
1393,16154,The world s two biggest greenhouse gas polluters have now ratified the Paris...,"[world, s, greenhouse, gas, polluters, governments, excuse, governments, fos..."
1394,16160,Regulators are rolling out a controversial pesticide banned in Europe to kil...,"[regulators, pesticide, mosquitoes, bees, bees, regulators, spraying, petiti..."
1395,16416,Dear Great news After 225000 of us spoke out against the mismanagement of...,"[news, mismanagement, water, water, rates, users, companies, milestone, towa..."
1396,16458,Foreign worker lists That s what this government wants to make all firms pu...,"[worker, lists, government, firms, plans, workers, workers, headline, days, ..."
1397,16479,Asos is coming under fire for treating its workers like machines Tell Aso...,"[fire, workers, machines, workers, petition, cost, fashion, giant, clothes, ..."
1398,16736,Corporations have the government s ear when it comes to Brexit it promised...,"[corporations, government, tariff, access, market, corporations, brexit, dea..."
1399,16940,Urgent The Trudeau government could approve the Kinder Morgan tar sands pip...,"[government, sands, pipeline, days, dozens, organizations, hundreds, thousan..."


### Up until this point, I have kept the text of mailings seperate from the tags, but at this point I want to grab the words from the 'tag_name' column and add them to the mailing text to be evaluated together.  I couldn't do that until after I ran the part of speech filter, because NLTK needs to have sentences in context in order to  properly mark the part of speech; adding short phrases would have created problems.  From here on in, I will be dealing with collections of words, where the order doesn't matter.  Because the tags were specifically chosen to convey topic information, I have chosen to give them 3x more weight than the regular mailing text, based on trial and error.

In [36]:
merged = pd.DataFrame(filtered[['page_id','text_clean','text_filtered','tags_clean']]) #copy results of last step into a new frame

def concat_cols(df, filtered_col, unfiltered_col, new_col, coef):
    weighted = ((df[unfiltered_col].str.lower()+' ')*coef) #multiply the unfiltered column by the desired coefficient
    df[new_col] = weighted.apply(lambda row: nltk.word_tokenize(row)) #tokenize (pre-process) each word 
    df[new_col] = df[new_col] + df[filtered_col]
    return df

merged['text_merged'] = '' # a new column to store the combined result of text and tags                     

merged = concat_cols(merged,'text_filtered','tags_clean','text_merged',3)

pd.options.display.max_colwidth =70
merged[['page_id','text_filtered','tags_clean','text_merged']].tail(10)

,page_id,text_filtered,tags_clean,text_merged
1390,16102,"[adverts, women, accommodation, sex, hundreds, vile, ads, petition...",Womens Rights,"[womens, rights, womens, rights, womens, rights, adverts, women, a..."
1391,16118,"[taxes, half, tax, evasion, petition, secret, technology, giant, p...",taxes Economic Justice trade,"[taxes, economic, justice, trade, taxes, economic, justice, trade,..."
1392,16148,"[world, s, greenhouse, gas, polluters, excuse, fossil, fuel, indus...",Environment,"[environment, environment, environment, world, s, greenhouse, gas,..."
1393,16154,"[world, s, greenhouse, gas, polluters, governments, excuse, govern...",Environment,"[environment, environment, environment, world, s, greenhouse, gas,..."
1394,16160,"[regulators, pesticide, mosquitoes, bees, bees, regulators, sprayi...",bees Environment,"[bees, environment, bees, environment, bees, environment, regulato..."
1395,16416,"[news, mismanagement, water, water, rates, users, companies, miles...",Environment,"[environment, environment, environment, news, mismanagement, water..."
1396,16458,"[worker, lists, government, firms, plans, workers, workers, headli...",Workers Rights,"[workers, rights, workers, rights, workers, rights, worker, lists,..."
1397,16479,"[fire, workers, machines, workers, petition, cost, fashion, giant,...",Workers Rights,"[workers, rights, workers, rights, workers, rights, fire, workers,..."
1398,16736,"[corporations, government, tariff, access, market, corporations, b...",Privatization and Political Meddling,"[privatization, and, political, meddling, privatization, and, poli..."
1399,16940,"[government, sands, pipeline, days, dozens, organizations, hundred...",oil company Environment,"[oil, company, environment, oil, company, environment, oil, compan..."


### Next I need to assemble a list of words that I want to exclude from the analysis. NLTK has gotten rid of many of the words we don't need based on parts of speech, but other words are specific to SumOfUs use, and have to be listed manually.  Words like 'petition', 'corporation', and 'click' are not salient for us, even though they could be salient in another context. I started with a list of generic stopwords and then added to it by hand.  Using the frequency distribution module of NLTK to make it easier, I can see the most commonly used words in my dataset, and then add the ones I want to exclude to my stoplist.  I ran this step several times until I was left with a list of salient terms.

In [37]:
go = pd.DataFrame(merged[['page_id','text_clean','text_merged']]) #copy results of last step into a new frame

def exclude_stopwords(df,old_col,new_col,num):
    stop_words = set(stopwords.words('english')) #read in my text file of stopwords   
    df[new_col] = df[old_col].apply(lambda x:[word for word in x if word not in stop_words]) #remove the stop words
    go_freq = FreqDist(df[new_col].sum()).most_common(num) #return the top words that are left by frequency
    return df, go_freq


go['text_go'] = ''
    
go, go_freq = exclude_stopwords(go,'text_merged','text_go',20)

go_freq #I use this list to find common words that I don't want in my analysis, then manually add them the stoplist

go_freq

[('oil', 1179),
 ('food', 974),
 ('water', 788),
 ('tax', 678),
 ('protection', 665),
 ('climate', 628),
 ('health', 521),
 ('liberties', 512),
 ('trade', 498),
 ('media', 469),
 ('economic', 459),
 ('privatization', 457),
 ('meddling', 438),
 ('women', 427),
 ('gmos', 372),
 ('customers', 355),
 ('children', 310),
 ('coal', 292),
 ('conditions', 283),
 ('farmers', 283)]

### Some words have the same root, such as 'work', 'working', and 'worker' or 'economy', 'economic' and 'economists.  For our purposes, these words have basically the same meaning, and should be combined in order to properly represent the weight of each term in the campaign.  I apply three different 'stemmer' algorithms, in order to use the roots of words in my model.

In [38]:
# stem the filtered tokens
stemmed = pd.DataFrame(go[['page_id','text_clean','text_go']]) #copy results of last step into a new frame

snowball = SnowballStemmer('english')
lancaster = LancasterStemmer()
porter = PorterStemmer()

def stem(df,old_col,new_col):
    df[new_col] = df[old_col].apply(lambda x: [porter.stem(word) for word in x])
    df[new_col] = df[new_col].apply(lambda x: [snowball.stem(word) for word in x])
    df[new_col] = df[new_col].apply(lambda x: [lancaster.stem(word) for word in x])
    return df

stemmed['text_stemmed'] = ''
stemmed = stem(stemmed,'text_go','text_stemmed')

pd.options.display.max_colwidth =90
stemmed[['page_id','text_go','text_stemmed']].tail(10)

,page_id,text_go,text_stemmed
1390,16102,"[womens, womens, womens, adverts, women, accommodation, sex, vile, bans, adverts, wome...","[wom, wom, wom, advert, wom, accommod, sex, vil, ban, advert, wom, exchang, sex, wom, ..."
1391,16118,"[taxes, economic, trade, taxes, economic, trade, taxes, economic, trade, taxes, tax, e...","[tax, econom, trad, tax, econom, trad, tax, econom, trad, tax, tax, ev, secret, techno..."
1392,16148,"[greenhouse, gas, polluters, excuse, fuel, climate, climate, treaty, force, treaty, fu...","[greenh, ga, pollut, exc, fuel, clim, clim, treat, forc, treat, fuel, clim, forc, clim..."
1393,16154,"[greenhouse, gas, polluters, governments, excuse, governments, fuel, climate, governme...","[greenh, ga, pollut, govern, exc, govern, fuel, clim, govern, govern, treat, fuel, cli..."
1394,16160,"[bees, bees, bees, regulators, pesticide, mosquitoes, bees, bees, regulators, spraying...","[bee, bee, bee, reg, pesticid, mosquito, bee, bee, reg, spray, mosquito, reg, pesticid..."
1395,16416,"[mismanagement, water, water, rates, milestone, water, ecology, economy, conversation,...","[mism, wat, wat, rat, mileston, wat, ecolog, econom, conv, anch, med, wat, approv, com..."
1396,16458,"[lists, firms, headline, flames, racism, businesses, workforce, lists, doctors, visa, ...","[list, firm, headlin, flam, rac, bus, workforc, list, doct, vis, stud, workforc, badg,..."
1397,16479,"[machines, fashion, clothes, expense, warehouse, warehouse, water, toilet, breaks, per...","[machin, fash, clo, exp, wareh, wareh, wat, toilet, break, perform, second, turnov, il..."
1398,16736,"[privatization, meddling, privatization, meddling, privatization, meddling, tariff, br...","[priv, meddl, priv, meddl, priv, meddl, tariff, brexit, strategi, min, car, auto, nego..."
1399,16940,"[oil, oil, oil, pipeline, tar, pipeline, megaproject, messages, pipeline, pipeline, co...","[oil, oil, oil, pipelin, tar, pipelin, megaproject, mess, pipelin, pipelin, controvers..."


### Now, at long last, I am finally able to get to the good stuff and build my model!  I use a module called gensim; the name is derived from “generate similar” because it generates topics based on the similarity of words within each group.  The first step is to take my painstakingly cleaned and filtered text and create a giant "bag of words", called a corpus, to feed into a Latent Dirichlet Allocation (LDA) model.  The model will derive topics from our corpus based on word frequency.   It then returns the top four words by frequency in each topic.  The coefficients represent the percentage of that word in the topic as a whole.  This alone does not tell you that much about the topics, so in the next step, I generate an interactive graphic to explore the topics in more depth.

In [9]:
def generate_lda(df,col,topics,words,num_passes):
    camp_corpus = df[col].tolist()
    dictionary = corpora.Dictionary(camp_corpus)
    corpus = [dictionary.doc2bow(text) for text in camp_corpus]
    lda = models.ldamodel.LdaModel(corpus, num_topics=topics, id2word=dictionary, passes=num_passes)
    topics = lda.print_topics(num_topics=topics,num_words=words)
    return lda, corpus, dictionary, topics

lda, corpus, dictionary, all_topics = generate_lda(stemmed,'text_stemmed', 8, 4, 150)

all_topics

[(0, '0.027*libert + 0.023*med + 0.023*protect + 0.020*prison'),
 (1, '0.076*wat + 0.046*priv + 0.044*meddl + 0.036*min'),
 (2, '0.122*oil + 0.037*palm + 0.028*forest + 0.021*deforest'),
 (3, '0.052*oil + 0.033*pipelin + 0.019*spil + 0.018*whal'),
 (4, '0.090*tax + 0.043*econom + 0.021*factor + 0.018*wag'),
 (5, '0.091*wom + 0.019*sex + 0.017*med + 0.013*viol'),
 (6, '0.083*food + 0.042*farm + 0.035*gmo + 0.034*bee'),
 (7, '0.048*trad + 0.048*clim + 0.024*coal + 0.024*bank')]

### Here I have generated an interactive visualization that will help me explore and understand each topic.  Each bubble represents a single topic. The size of the bubble indicating the relative proportion of 'tokens' (words) in the corpus that are related to that topic.  The bubbles are numbered by size in descending order and do not match the topic listing in the previous step.

### The topics are placed on a field with two axes: PC1 and PC2.  PC stands for Principal Component. Principal component analysis (PCA) is a technique used to emphasize variation and bring out strong patterns in a dataset. It's often used to make data easy to explore and visualize. The axes don't mean any one thing but represent a composite of features chosen algorithmically to maximize variation.  Which is to say, the placement of the bubbles on the field shows you how closely related the topics are to each other in terms of the words they contain.  

### A single word can be found in more than one topic when it used in different contexts.  For example, the word 'oil' often appears with 'pipeline', and also often appears with 'palm'.  The algorithm makes guesses as to what topic a word belong to, based on other words in the same campaigns.  When two topics share many terms they are shown on the field as overlapping.

###  The blue bars on the right represent the overall frequency of each term in the corpus.  If you mouse over any bubble, red bars appear to the right.  The red bars represent the estimated number of times a given term was included in a given topic.   If you mouse over any word on the right, the bubbles change size to show the be proportional to the frequency of that term.

In [24]:
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

### Now that the topics have been generated, I need to apply them back to the campaigns.  The LDA algorithm looks at each campaign, and guesses a percentage chance that campaign is related to each of the topics.  Note that the topic numbers here related to the original numbered list, not the numbers on the bubbles in the visualization. 


In [41]:
assigned = pd.DataFrame(stemmed[['page_id','text_clean','text_stemmed']])  #copy results of last step into a new frame

def calc_freq(df,old_col,new_col,num):
    for index, row in df.iterrows():
        camp_freq = df['text_stemmed'][index]
        dist = FreqDist(camp_freq)
        df.set_value(index,'freq',dist.most_common(num))
    return df

def assign_topics(df,old_col,new_col,num):
    for index, row in df.iterrows():
        seed = df[old_col][index]
        doc_bow = dictionary.doc2bow(seed)
        camp_tops = lda.get_document_topics(doc_bow,num)
        df.set_value(index,new_col, camp_tops)
    return df
                              
assigned['freq'] = ''
assigned['topics'] = ''

assigned = calc_freq(assigned,'text_stemmed','freq',20)
assigned = assign_topics(assigned,'text_stemmed','topics',0).reset_index(drop=True)

pd.options.display.max_colwidth =150
assigned[['page_id','freq','topics']].tail(10)

,page_id,freq,topics
1390,16102,"[(wom, 8), (advert, 5), (sex, 4), (accommod, 3), (exchang, 2), (vil, 2), (pic, 2), (rent, 2), (homeless, 2), (resourc, 1), (slaver, 1), (ban, 1), ...","[(0, 0.00256041641382), (1, 0.00255410743216), (2, 0.0025537692352), (3, 0.00255205065976), (4, 0.00255502800512), (5, 0.982119981945), (6, 0.0025..."
1391,16118,"[(tax, 11), (econom, 3), (trad, 3), (period, 2), (investig, 2), (account, 2), (tril, 1), (dodg, 1), (merg, 1), (aid, 1), (ev, 1), (offend, 1), (ar...","[(0, 0.00305359287891), (1, 0.00305090224483), (2, 0.00305159881984), (3, 0.00305368670032), (4, 0.740677380524), (5, 0.132971964313), (6, 0.00305..."
1392,16148,"[(clim, 5), (fuel, 4), (treat, 3), (forc, 3), (pollut, 2), (emiss, 2), (med, 2), (pledg, 1), (govern, 1), (feet, 1), (glob, 1), (progress, 1), (ex...","[(0, 0.00379604140077), (1, 0.0037902018706), (2, 0.00378973743449), (3, 0.00379002167795), (4, 0.00378924972793), (5, 0.00379355203538), (6, 0.00..."
1393,16154,"[(govern, 8), (fuel, 4), (clim, 4), (med, 2), (treat, 2), (emiss, 2), (pollut, 2), (forc, 2), (pledg, 1), (glob, 1), (fal, 1), (exc, 1), (breakthr...","[(0, 0.00338558501494), (1, 0.0033801074324), (2, 0.00337947847134), (3, 0.00338011199237), (4, 0.00337966888893), (5, 0.00338278476308), (6, 0.00..."
1394,16160,"[(bee, 9), (reg, 7), (mosquito, 3), (chem, 3), (beekeep, 3), (pollin, 2), (spray, 2), (pesticid, 2), (liv, 2), (truck, 1), (issu, 1), (butterfl, 1...","[(0, 0.00250752800231), (1, 0.00250562613993), (2, 0.00250212911205), (3, 0.00250315525219), (4, 0.00250336061968), (5, 0.00250703835714), (6, 0.9..."
1395,16416,"[(wat, 6), (rat, 2), (conv, 1), (econom, 1), (legisl, 1), (anch, 1), (backlash, 1), (cla, 1), (ecolog, 1), (mism, 1), (commit, 1), (approv, 1), (m...","[(0, 0.102159034935), (1, 0.719122638206), (2, 0.00596803279105), (3, 0.148901019065), (4, 0.00596696699308), (5, 0.00595937240072), (6, 0.0059554..."
1396,16458,"[(crim, 5), (hat, 4), (rac, 3), (list, 2), (street, 2), (workforc, 2), (vis, 1), (tid, 1), (newsread, 1), (sham, 1), (stud, 1), (migr, 1), (firm, ...","[(0, 0.339403481374), (1, 0.00272251905347), (2, 0.00271904304297), (3, 0.00271949545065), (4, 0.00272164418958), (5, 0.492688085627), (6, 0.00271..."
1397,16479,"[(wareh, 4), (fash, 3), (factor, 2), (break, 1), (tax, 1), (second, 1), (perform, 1), (target, 1), (sal, 1), (clo, 1), (not, 1), (dut, 1), (toilet...","[(0, 0.00320891633333), (1, 0.00321975956164), (2, 0.00320747865582), (3, 0.21372889578), (4, 0.735038375679), (5, 0.00320716019355), (6, 0.003206..."
1398,16736,"[(priv, 3), (meddl, 3), (effect, 2), (tax, 2), (taxpay, 2), (strategi, 2), (sort, 2), (deb, 1), (reg, 1), (backroom, 1), (pocket, 1), (negot, 1), ...","[(0, 0.00284325556916), (1, 0.289432468902), (2, 0.00284847867454), (3, 0.0676561363346), (4, 0.320574683308), (5, 0.00284237209504), (6, 0.002845..."
1399,16940,"[(pipelin, 8), (oil, 3), (govern, 3), (street, 2), (bord, 1), (consequ, 1), (rout, 1), (cons, 1), (resist, 1), (mess, 1), (pri, 1), (controvers, 1...","[(0, 0.00297950635302), (1, 0.00298146097439), (2, 0.00297790434521), (3, 0.979146411637), (4, 0.00297900891586), (5, 0.00297864259201), (6, 0.002..."


### Now I need to break down that topics field so that each topic gets assigned to its own column. For readibility, I will assign each column a name, and convert the numbers into rounded percentages. Finally, I will both display the final table and save it as a csv for use in my linear regression.  The linear regresssion model will determine how predictive each topic is of a cohort's propensity to donate.  These topics will be combined with other information from the campaign like regional breakdown, campaign virality, and early behavior of new joiners from that cohort.  All of the information will be used to determine the best way to predict the probability that a new member from a given campaign will become a donor.

In [43]:
pivoted = pd.DataFrame(assigned[['page_id','text_clean','freq','topics']]) 
                                  
def pivot(df,topic_col,headers):
    topicframe = pd.DataFrame()
    topicframe[topic_col] = [[tuple[1] for tuple in topic] for topic in df[topic_col]]  #removes a layer of nesting
    topicframe = pd.DataFrame(topicframe[topic_col].tolist())
    topicframe = round(topicframe*100,0)
    topicframe.columns=headers
    
    df = pd.concat([df, topicframe], axis=1)
    return df

topic_headers = ['health','private','palm','fossil','econ','rights','food','trade']

pivoted = pivot(pivoted,'topics',topic_headers)

pivoted.to_csv('mailing_topic.csv')

pd.options.display.max_colwidth =70
#pivoted[['page_id','topics']+topic_headers]
pivoted.tail(10)

,page_id,text_clean,freq,topics,health,private,palm,fossil,econ,rights,food,trade
1390,16102,Craigslist allow exploitative adverts offering homeless women acco...,"[(wom, 8), (advert, 5), (sex, 4), (accommod, 3), (exchang, 2), (vi...","[(0, 0.00256041641382), (1, 0.00255410743216), (2, 0.0025537692352...",0.0,0.0,0.0,0.0,0.0,98.0,0.0,0.0
1391,16118,Apple is avoiding more than 13 billion in taxes And that s not e...,"[(tax, 11), (econom, 3), (trad, 3), (period, 2), (investig, 2), (a...","[(0, 0.00305359287891), (1, 0.00305090224483), (2, 0.0030515988198...",0.0,0.0,0.0,0.0,74.0,13.0,0.0,11.0
1392,16148,The world s two biggest greenhouse gas polluters have now ratified...,"[(clim, 5), (fuel, 4), (treat, 3), (forc, 3), (pollut, 2), (emiss,...","[(0, 0.00379604140077), (1, 0.0037902018706), (2, 0.00378973743449...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.0
1393,16154,The world s two biggest greenhouse gas polluters have now ratified...,"[(govern, 8), (fuel, 4), (clim, 4), (med, 2), (treat, 2), (emiss, ...","[(0, 0.00338558501494), (1, 0.0033801074324), (2, 0.00337947847134...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0
1394,16160,Regulators are rolling out a controversial pesticide banned in Eur...,"[(bee, 9), (reg, 7), (mosquito, 3), (chem, 3), (beekeep, 3), (poll...","[(0, 0.00250752800231), (1, 0.00250562613993), (2, 0.0025021291120...",0.0,0.0,0.0,0.0,0.0,0.0,98.0,0.0
1395,16416,Dear Great news After 225000 of us spoke out against the misman...,"[(wat, 6), (rat, 2), (conv, 1), (econom, 1), (legisl, 1), (anch, 1...","[(0, 0.102159034935), (1, 0.719122638206), (2, 0.00596803279105), ...",10.0,72.0,1.0,15.0,1.0,1.0,1.0,1.0
1396,16458,Foreign worker lists That s what this government wants to make al...,"[(crim, 5), (hat, 4), (rac, 3), (list, 2), (street, 2), (workforc,...","[(0, 0.339403481374), (1, 0.00272251905347), (2, 0.00271904304297)...",34.0,0.0,0.0,0.0,0.0,49.0,0.0,15.0
1397,16479,Asos is coming under fire for treating its workers like machines ...,"[(wareh, 4), (fash, 3), (factor, 2), (break, 1), (tax, 1), (second...","[(0, 0.00320891633333), (1, 0.00321975956164), (2, 0.0032074786558...",0.0,0.0,0.0,21.0,74.0,0.0,0.0,4.0
1398,16736,Corporations have the government s ear when it comes to Brexit i...,"[(priv, 3), (meddl, 3), (effect, 2), (tax, 2), (taxpay, 2), (strat...","[(0, 0.00284325556916), (1, 0.289432468902), (2, 0.00284847867454)...",0.0,29.0,0.0,7.0,32.0,0.0,0.0,31.0
1399,16940,Urgent The Trudeau government could approve the Kinder Morgan tar...,"[(pipelin, 8), (oil, 3), (govern, 3), (street, 2), (bord, 1), (con...","[(0, 0.00297950635302), (1, 0.00298146097439), (2, 0.0029779043452...",0.0,0.0,0.0,98.0,0.0,0.0,0.0,0.0
